Structured variational message passing demo
===

A generalization of VMP under the mean field approximation is structured VMP (SVMP). SVMP allows arbitrary factorizations of the approximating $q$ distribution (Dauwels 2007). As an example case to illustrate and test our SVMP implementation in ForneyLab, we estimate the mean $m$ and precision $\gamma$ for a set of samples $y = \{y_1, ..., y_k, ..., y_n\}$ drawn from a Gaussian distribution.

The factor graph below shows our generative model, where we use the notation of (Reller, 2012; State-space methods in statistical signal processing). Filled black nodes represent observed variables and dotted arcs denote the repetition of a section.

<img src="images/gauss_est_prec.png">

This is the same model as in the previous demo, however this time we use precision parametrization and factorize the approximating distribution as $q(m,\gamma,y)=q(m,\gamma)\,q(y)$. Also, we estimate the mean and precision in a sample-by sample (online) fashion instead of in bulk.

We begin by implementing the factor graph in ForneyLab and define the structured factoriation.

In [1]:
using(ForneyLab)

# Initialize nodes and edges
GaussianNode(id=:g, form=:precision)
EqualityNode(id=:m_eq) # Equality node for mean
EqualityNode(id=:gam_eq) # Equality node for precision
TerminalNode(id=:y) # Observed y values are stored in y_node
PriorNode(GaussianDistribution(m=0.0, V=100.0), id=:m_n_min_1) # Prior
PriorNode(GammaDistribution(a=0.01, b=0.01), id=:gam_n_min_1) # Prior
TerminalNode(vague(GaussianDistribution), id=:m_n)
TerminalNode(vague(GammaDistribution), id=:gam_n)

Edge(n(:g).i[:out], n(:y), GaussianDistribution)
Edge(n(:m_eq).i[3], n(:g).i[:mean], GaussianDistribution)
Edge(n(:gam_eq).i[3], n(:g).i[:precision], GammaDistribution)
Edge(n(:m_n_min_1), n(:m_eq).i[1], GaussianDistribution)
Edge(n(:gam_n_min_1), n(:gam_eq).i[1], GammaDistribution)
Edge(n(:m_eq).i[2], n(:m_n), GaussianDistribution)
Edge(n(:gam_eq).i[2], n(:gam_n), GammaDistribution)

# Set wraps
Wrap(n(:m_n), n(:m_n_min_1))
Wrap(n(:gam_n), n(:gam_n_min_1));

Now we need to specify the factorization, update schedules for both subgraphs and set the vague (almost uninformative) marginals.

In [2]:
# Generate some data
true_mean = 5.0
true_prec = 0.5
n_samples = 100 # Draw 100 samples
n_its = 10 # Iterations per data sample
y_vals = randn(n_samples)*(1/sqrt(true_prec))+true_mean
data = [DeltaDistribution(y) for y in y_vals]

# Add read and write buffers
y_buffer = setReadBuffer(n(:y), data)
m_est = setWriteBuffer(n(:m_eq).i[2])
gam_est = setWriteBuffer(n(:gam_eq).i[2])

# Define the structured variational algorithm
algo = VMP.Algorithm(Set{Edge}({n(:g).i[:out].edge}), n_iterations=n_its)

# Inspect the factorization (optional)
for subgraph in algo.fields[:factorization].factors
    draw(subgraph)
end

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 6884156079908955013 -->
 6884156079908955013 
 
 GaussianNode 
 g 
 
<!-- 8346919566901068157 -->
 8346919566901068157 
 
<!-- 6884156079908955013->8346919566901068157 -->
 6884156079908955013->8346919566901068157 
 
 
 1 out 
 3 out 
 
<!-- 731040640712094714 -->
 731040640712094714 
 
 m_n 
 
<!-- 13239938299758760890 -->
 13239938299758760890 
 
 gam_n_min_1 
 
<!-- 5832917416748456464 -->
 5832917416748456464 
 
 = 
 gam_eq 
 
<!-- 13239938299758760890->5832917416748456464 -->
 13239938299758760890->5832917416748456464 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GammaDistribution 
 1 1 
 1 out 
 
<!-- 5832917416748456464->6884156079908955013 -->
 5832917416748456464->6884156079908955013 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GammaDistribution 
 2 precision 
 3 3 
 
<!-- 6261009887430528463 -->
 6261009887430528463 
 
 gam_n 
 
<!-- 5832917416748456464->6261009887430528463 -->
 5832917416748456464->6261009887430528463 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GammaDistribution 
 1 out 
 2 2 
 
<!-- 8620245085929121030 -->
 8620245085929121030 
 
 = 
 m_eq 
 
<!-- 8620245085929121030->6884156079908955013 -->
 8620245085929121030->6884156079908955013 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GaussianDistribution 
 1 mean 
 3 3 
 
<!-- 8620245085929121030->731040640712094714 -->
 8620245085929121030->731040640712094714 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GaussianDistribution 
 1 out 
 2 2 
 
<!-- 13764711498252298853 -->
 13764711498252298853 
 
 m_n_min_1 
 
<!-- 13764711498252298853->8620245085929121030 -->
 13764711498252298853->8620245085929121030 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GaussianDistribution 
 1 1 
 1 out

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 6884156079908955013 -->
 6884156079908955013 
 
 GaussianNode 
 g 
 
<!-- 8346919566901068157 -->
 8346919566901068157 
 
 y 
 
<!-- 6884156079908955013->8346919566901068157 -->
 6884156079908955013->8346919566901068157 
 
 
 FW: ○ 
 BW: ○ 
 Distribution: GaussianDistribution 
 1 out 
 3 out 
 
<!-- 5832917416748456464 -->
 5832917416748456464 
 
<!-- 5832917416748456464->6884156079908955013 -->
 5832917416748456464->6884156079908955013 
 
 
 2 precision 
 3 3 
 
<!-- 8620245085929121030 -->
 8620245085929121030 
 
<!-- 8620245085929121030->6884156079908955013 -->
 8620245085929121030->6884156079908955013 
 
 
 1 mean 
 3 3

Now the setup is done and we can start the iterations.

In [3]:
run(algo)

Let's inspect the results.

In [4]:
println("True mean: $(true_mean)")
println("True precision: $(true_prec)")
println("Number of samples: $(n_samples)")
println("Sample mean: $(round(mean(y_vals),2))")
println("Sample precision: $(round(1/var(y_vals),2))")
println("\n----- Online estimation after $(n_its) VMP updates per sample -----")
println("Mean estimate: $(round(mean(m_est[end])[1],2)), with variance $(round(var(m_est[end])[1,1],2))")
println("Precision estimate: $(round(mean(gam_est[end]),2)), with variance $(round(var(gam_est[end]),2))")

True mean: 5.0


True precision: 0.5
Number of samples: 100
Sample mean: 4.71
Sample precision: 0.59

----- Online estimation after 10 VMP updates per sample -----
Mean estimate: 4.69, with variance 0.03
Precision estimate: 0.49, with variance 0.0
